## Models, Prompts and Parsers
Based on the "LangChain for LLM Application Development" by DeepLearning and Langchain

### Try 1
Normal 

In [1]:
from langchain_community.llms import Ollama

model = Ollama(model="llama3.1:latest", temperature=0, base_url="https://b0b1-34-125-172-82.ngrok-free.app/")


In [2]:
def get_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = model.invoke(messages)
    return response

In [3]:
get_completion("What is 1+1?")

'The answer to that question is... 2!'

In [4]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [5]:
style = """American English \
in a calm and respectful tone
"""

In [6]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
print(get_completion(prompt))

Here is the translated text in American English:

"I'm really frustrated that my blender lid came loose and spilled a smoothie all over my kitchen walls! To make things even more inconvenient, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now."


### Try 2
Creating custom reuseable prompts from langchain's ChatPromptTemplate

In [8]:
from langchain_community.llms import Ollama

model = Ollama(model="llama3.1:latest", temperature=0, base_url="https://b0b1-34-125-172-82.ngrok-free.app/")
model

Ollama(base_url='https://b0b1-34-125-172-82.ngrok-free.app/', model='llama3.1:latest', temperature=0.0)

In [9]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [10]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [11]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [12]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [13]:
customer_style = """American English \
in a calm and respectful tone
"""

In [14]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [15]:
customer_messages = prompt_template.format_messages(style=customer_style,text=customer_email)

In [16]:
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [17]:
customer_response = model.invoke(customer_messages)

In [18]:
print(customer_response)

Here is the translated text in a calm and respectful tone, using American English:

"I'm really frustrated that my blender lid flew off and spilled smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now."


In [19]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [20]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [21]:
service_messages = prompt_template.format_messages(style=service_style_pirate,text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [22]:
service_response = model.invoke(service_messages)

In [23]:
print(service_response)

Here be the translated text, me hearty:

```Ahoy, valued customer!

We regret to inform ye that yer warranty don't cover the cost o' cleanin' up the mess ye made in yer kitchen. It seems ye had a bit o' a mishap with yer trusty blender, and it's clear ye didn't follow proper procedure by forgettin' to put the lid on before turnin' it on.

We know it be a disappointment, but we can't make an exception just this once. Perhaps ye should take heed o' this lesson and be more mindful o' yer kitchen duties in the future?

Farewell for now, and may the winds o' fortune blow in yer favor!```


### Try 3
Creating custom prompt to provide prased output [JSON] (output is indeed json style but not json format)

In [24]:
from langchain_community.llms import Ollama

model = Ollama(model="llama3.1:latest", temperature=0, base_url="https://b0b1-34-125-172-82.ngrok-free.app/")
model

Ollama(base_url='https://b0b1-34-125-172-82.ngrok-free.app/', model='llama3.1:latest', temperature=0.0)

In [25]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [26]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [27]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [28]:
messages = prompt_template.format_messages(text=customer_review)

In [29]:
response = model.invoke(messages)

In [30]:
print(response)

Here is the extracted information in JSON format:

```
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


### Try 4
Trying out ResponseSchema and StrucredOutputParser to get JSON formated output, but it didnt go as planned, the tutorial got the right answer but not me

In [31]:
from langchain_community.llms import Ollama

model = Ollama(model="llama3.1:latest", temperature=0, base_url="https://b0b1-34-125-172-82.ngrok-free.app/")
model

Ollama(base_url='https://b0b1-34-125-172-82.ngrok-free.app/', model='llama3.1:latest', temperature=0.0)

In [32]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [33]:
gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [34]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [35]:
format_instructions = output_parser.get_format_instructions()

In [36]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [38]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [39]:
messages[0].content

'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "

In [40]:
response = model.invoke(messages)

In [41]:
print(response)

Here is the extracted information in the requested format:

```json
{
    "gift": True,
    "delivery_days": "2",
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```

Note that I've formatted the output as JSON, with string values enclosed in double quotes. The `gift` field is set to True because the text mentions that the item was purchased "just in time for my wife's anniversary present", implying that it was a gift. The `delivery_days` field is set to 2, which is the number of days mentioned in the text as taking for the product to arrive. The `price_value` field contains the sentence about the price from the text.


In [46]:

output_dict = output_parser.parse(response)

OutputParserException: Got invalid JSON object. Error: Expecting value: line 2 column 13 (char 14)